In [1]:
import networkx as nx
from itertools import islice
import random
from itertools import groupby
import time
import math as mt
import csv
import os
import random
import math
import pdb

from os import listdir

from os.path import isfile, join

In [11]:
def create_2D_graph(SURFnet_topology,size_of_dimention):
    link_lenghts = []
    try:
        f = open(SURFnet_topology+".txt", 'r')
    except:
        f = open(SURFnet_topology, 'r')
    header = f.readline()
    nodes = []
    flows = []
    real_links_rates = []
    for line in f:
        line = line.strip()
        link = line.split('\t')
        real_virtual = "real"
        try:
            i, s, d,  c,l,real_virtual = link
        except:
            i, s, d,  c,l = link
        if real_virtual =="real":
            link_lenghts.append(round(float(l),2))
            real_links_rates.append(float(c))
    G = nx.grid_2d_graph(size_of_dimention, size_of_dimention)  # size_of_dimention x size_of_dimention grid
    edge_indx = 0
    lines = ["Link_index	Source	Destination	Capacity(EPRps)	Length(km)	Real/Virtual"+"\n"]
    node_counter = 0
    each_2d_node_node_id ={}
    for edge in G.edges:
        try:
            if edge[0] not in each_2d_node_node_id:
                each_2d_node_node_id[edge[0]] = node_counter
                node_counter+=1
        except:
            each_2d_node_node_id[edge[0]] = node_counter
            node_counter+=1
        try:
            if edge[1] not in each_2d_node_node_id:
                each_2d_node_node_id[edge[1]] = node_counter
                node_counter+=1
        except:
            each_2d_node_node_id[edge[1]] = node_counter
            node_counter+=1
            
    node_max_id=node_counter
    for edge in G.edges:
        print("this is edge ",edge)
        
        node1 = each_2d_node_node_id[edge[0]]
        node2 = each_2d_node_node_id[edge[1]]
        distance = link_lenghts[random.randint(0,len(link_lenghts)-1)]
        c = 1
        etha = 10**(-0.1*0.2*distance)
        T = (distance*10**(-4))/25# based on simulation setup of data link layer paper
        if T==0:
            T=1 * 10**(-6)
        alpha = 1
        #print("T is %s for length %s"%(T,distance))
        distance = round(distance,3)
        edge_rate = round((2*c*etha*alpha)/T,3)
        line = str(edge_indx)+"\t"+str(node1)+"\t"+str(node2)+"\t"+str(edge_rate)+"\t"+str(distance)+"\t"+"real"+"\n"
        edge_indx+=1
        lines.append(line)
        for i in range(3):
            new_node1 = node_max_id
            node_max_id = node_max_id+1
            edge_rate = max(real_links_rates)+100
            
            line = str(edge_indx)+"\t"+str(node1)+"\t"+str(new_node1)+"\t"+str(edge_rate)+"\t"+str(0)+"\t"+"virtual"+"\n"
            edge_indx+=1
            lines.append(line)

            #distance = round(random.uniform(min(all_distances),max(all_distances)),2)
            
            edge_rate = edge_rate
            line = str(edge_indx)+"\t"+str(new_node1)+"\t"+str(node2)+"\t"+str(edge_rate)+"\t"+str(distance)+"\t"+"real"+"\n"
            edge_indx+=1
            lines.append(line)
            

    with open("data/2D.txt", "a+") as file_object:
        for line in lines:
            file_object.write(line)


In [3]:
#create_2D_graph("data/ModifiedSurfnet.txt",10)

In [ ]:
pur_schemes = ["end_level","edge_level"]
pur_schemes.sort(reverse=False)
print(pur_schemes)

In [ ]:
def split(x, n):
    points = []
    # If we cannot split the
    # number into exactly 'N' parts
    if(x < n):
        print(-1)
 
    # If x % n == 0 then the minimum
    # difference is 0 and all
    # numbers are x / n
    elif (x % n == 0):
        for i in range(n):
#             print(x//n, end =" ")
            points.append(x//n)
    else:
        # upto n-(x % n) the values
        # will be x / n
        # after that the values
        # will be x / n + 1
        zp = n - (x % n)
        pp = x//n
        for i in range(n):
            if(i>= zp):
#                 print(pp + 1, end =" ")
                points.append(pp + 1)
            else:
#                 print(pp, end =" ")
                points.append(pp)
    return points
       



def parsing_zoo_topologies(target_original_network,target_link_lenght,dividing_link_metric):
    onlyfiles = [f for f in listdir("data/") if isfile(join("data/", f))]
    not_connected = 0
    connected = 0
    for file in onlyfiles:
        # computing distance between two nodes in graphgml data structure
        #print("this is the file ",file)
        if "graphml" in file and ( target_original_network in file and "Modified" not in file):
            G = nx.read_graphml("data/"+file)
            if nx.is_connected(G):
                lines = ["Link_index	Source	Destination	Capacity(EPRps)	Length(km)	Real/Virtual"+"\n"]
                randomly_selected_nodes = []
                each_real_link_lenght = {}
                each_real_link_rate = {}
                real_links_rates = []
                connected+=1
                latitude_values = []
                longitude_values = []
                degrees = []
                covered_edges = []
                diameter = nx.diameter(G)

                distances = []
                latitudes = nx.get_node_attributes(G,"Latitude")

                Longitudes  = nx.get_node_attributes(G,"Longitude")
    #             if file =="Darkstrand.graphml":
    #                 print("Longitudes",Longitudes)
    #                 print("latitudes",latitudes)
                counter = 0
                import haversine as hs
                nodes = []
                for node in G.nodes:
                    
                    nodes.append(int(node))
                    #print(node)
                    degrees.append(G.degree[node])
                    try:
                        #print("node %s latitudes %s Longitudes %s"%(node, latitudes[node],Longitudes[node]))
                        longitude_values.append(Longitudes[node])
                        latitude_values.append(latitudes[node])
                    except:
                        counter+=1
                        pass
                computed_distances = []
                for edge in G.edges:
                    flag = True
                    try:
                        coords_1 = (latitudes[edge[0]],Longitudes[edge[0]])
                    except:
                        flag = False
                    try:
                        coords_2 = (latitudes[edge[1]],Longitudes[edge[1]])
                    except:
                        flag = False
                    if flag:
                        distance = hs.haversine(coords_1,coords_2)
                        computed_distances.append(distance)
                edge_indx = 0
                node_max_id = max(nodes)+1
                real_edges = []
                all_distances = []
                new_repeaters_id = max(nodes)+1
                print("we have max id %s for nodes and out new repeater %s "%(max(nodes),new_repeaters_id))
                for edge in G.edges:
                    flag = True
                    try:
                        coords_1 = (latitudes[edge[0]],Longitudes[edge[0]])
                    except:
                        flag = False
                    try:
                        coords_2 = (latitudes[edge[1]],Longitudes[edge[1]])
                    except:
                        flag = False
                    if flag:
                        distance = hs.haversine(coords_1,coords_2)
                        if distance ==0:
                            print("computed distance is ",distance,coords_1,coords_2)
                            distance = 1
#                         else:
#                             print("one reasonable result")
                    else:
                        distance = round(random.uniform(min(computed_distances),max(computed_distances)),3)
                        #print("a randomly generated among %s %s %s"%(min(computed_distances),max(computed_distances),distance))
                    distance = round(distance,3)
                    print("%s ------------%s----------->%s "%(edge[0],distance,edge[1]))
                    if int(distance)<=target_link_lenght:
                        new_distance=distance
                        c = 1
                        etha = 10**(-0.1*0.2*new_distance)
                        T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
                        if T==0:
                            T=1 * 10**(-6)
                        alpha = 1
                        all_distances.append(new_distance)
                        edge_rate = round((2*c*etha*alpha)/T,3)
                        real_links_rates.append(edge_rate)
                        each_real_link_rate[(int(edge[0]),int(edge[1]))] = edge_rate
                        each_real_link_rate[(int(edge[1]),int(edge[0]))] = edge_rate
                        each_real_link_lenght[(int(edge[0]),int(edge[1]))] = new_distance
                        each_real_link_lenght[(int(edge[1]),int(edge[0]))] = new_distance
                        real_edges.append((int(edge[0]),int(edge[1])))
                        
                        print("unchanged link %s --> %s with length %s "%(edge[0],edge[1],new_distance))
                        line = str(edge_indx)+"\t"+str(edge[0])+"\t"+str(edge[1])+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
                        edge_indx+=1
                        lines.append(line)
                    else:
                    
                        n = int(math.ceil(distance/target_link_lenght))
                        list_points = split(distance, n)
                        flag = False
                        source = int(edge[0])
                        end = new_repeaters_id
                        flag = True
                        
                        for indx,point in enumerate(list_points):
                            new_distance = point
                            if flag:
                                flag = False
                                source = int(edge[0])
                                end = new_repeaters_id
                                print("connecting %s --> %s length %s"%(source,end,point))
                                new_repeaters_id+=1
                            else:
                                if indx== len(list_points)-1:
                                    source = end
                                    end = int(edge[1])
                                    print("connecting %s --> %s length %s"%(source,end,point))
                                    new_repeaters_id+=1
                                else:
                                    source = end
                                    end = new_repeaters_id
                                    print("connecting %s --> %s length %s"%(source,end,point))
                                    new_repeaters_id+=1
                        
                            c = 1
                            etha = 10**(-0.1*0.2*new_distance)
                            T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
                            if T==0:
                                T=1 * 10**(-6)
                            alpha = 1
                            all_distances.append(new_distance)
                            edge_rate = round((2*c*etha*alpha)/T,3)
                            real_links_rates.append(edge_rate)
                            each_real_link_rate[(source,end)] = edge_rate
                            each_real_link_rate[(end,source)] = edge_rate
                            each_real_link_lenght[(source,end)] = new_distance
                            each_real_link_lenght[(end,source)] = new_distance
                            real_edges.append((source,end))
                            line = str(edge_indx)+"\t"+str(source)+"\t"+str(end)+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
                            edge_indx+=1
                            lines.append(line)
                        
                        

                            
                    
                covered_edges = []
                added_edges = 0
                node_max_id = new_repeaters_id+1
                for edge in real_edges:
                    print("edge ",edge)
                    range_distance = max(all_distances) 
                    edge_distance = each_real_link_lenght[edge]
                   
                    paralel_links = 3
                    print(" edge %s with length %s range_distance %s "%(edge,edge_distance,range_distance
                                                                              ))
                    for i in range(paralel_links):
                        
#                         distance = round(random.uniform(edge_distance/2,3/2 *edge_distance),2)
                        distance= edge_distance
                        node1=edge[0]
                        node2=edge[1]
                        new_node1 = node_max_id
                        added_edges+=1
                        node_max_id = node_max_id+1
                        covered_edges.append((node1,node2))
                        edge_rate = max(real_links_rates)+100
                        distance = 0
                        line = str(edge_indx)+"\t"+str(node1)+"\t"+str(new_node1)+"\t"+str(edge_rate)+"\t"+str(distance)+"\t"+"virtual"+"\n"
                        edge_indx+=1
                        lines.append(line)

                        #distance = round(random.uniform(min(all_distances),max(all_distances)),2)
                        distance=edge_distance
                        edge_rate = each_real_link_rate[edge]
                        line = str(edge_indx)+"\t"+str(new_node1)+"\t"+str(node2)+"\t"+str(edge_rate)+"\t"+str(distance)+"\t"+"real"+"\n"
                        edge_indx+=1
                        lines.append(line)

                            
                            
                     # Open the file in append & read mode ('a+')
                print("this is file",file)
                file_name = file.split(".graphml")[0]
                if os.path.exists("data/Modified"+target_original_network+".txt"):
                    os.remove("data/Modified"+target_original_network+".txt")
                with open("data/Modified"+target_original_network+".txt", "a+") as file_object:
                    for line in lines:
                        file_object.write(line)
#                 print("network %s \n has %s nodes \n %s edges \n avg degree %s \n diameter %s \n avg distance %s \n min distance %s \n max distance %s \n unfound nodes %s"%(file,len(G.nodes),len(G.edges), sum(degrees)/len(degrees),diameter, sum(distances)/len(distances),min(distances),max(distances),counter))
#                 print("%s ,  %s ,  %s , , %s , %s , %s , %s , %s "%(file,len(G.nodes),len(G.edges), sum(degrees)/len(degrees),diameter, sum(distances)/len(distances),min(distances),max(distances)))
            else:
                not_connected+=1
            #return 
    print("max lenght %s acg %s min %s "%(max(all_distances),sum(all_distances)/len(all_distances),min(all_distances)))

In [ ]:
# parsing_zoo_topologies("Surfnet",20,10)


In [2]:
def distance_between_two_nodes(graph,source,target,virtual_links):
    path_length = 0
    path =  nx.shortest_path(graph, source, target)
    for edge in path:
        if edge not in virtual_links:
            path_length+=1
    return path_length
def work_load_generator(original_topology_name,topology_file,number_of_work_loads,organization_num,list_of_number_of_flows,hops_between_end_nodes):
    global each_flow_size_upper_flow_size
    global number_of_organziation 
    global lower_bound_Fth
    global upper_bound_Fth 
    global flow_weight_lower_boud
    global flow_weight_upeer_bound 


    onlyfiles = [f for f in listdir("data/") if isfile(join("data/", f))]
    not_connected = 0
    connected = 0
    for file in onlyfiles:
        if "graphml" in file and ( original_topology_name in file and "Modified" not in file):
            G = nx.read_graphml("data/"+file)
            if nx.is_connected(G):
                selected_real_nodes = []
                for node in G.nodes:
                    selected_real_nodes.append(int(node))
    
    
    virtual_links = []
    g = nx.Graph()
    print('[*] Loading topology...', topology_file)
    
    try:
        f = open(topology_file+".txt", 'r')
    except:
        f = open(topology_file, 'r')
    header = f.readline()
    virtual_nodes = []
    for line in f:
        line = line.strip()
        link = line.split('\t')
        #print(line,link)
        real_virtual = "real"
        try:
            i, s, d,  c,l,real_virtual = link
        except:
            i, s, d,  c,l = link
        if real_virtual =="virtual":
            if (int(s),int(d)) not in virtual_links:
                virtual_links.append((int(s),int(d)))
            if int(s) not in virtual_nodes:
                virtual_nodes.append(int(s))
            
    
    WK2lines = []
    lines = []
    work_load_file_lines = []
    sum_of_weights ={}
    each_wk_k_weight = {}
    for wk_idx in range(number_of_work_loads):
        for organization in range(organization_num):
            org_weight = round(random.uniform(flow_weight_lower_boud,flow_weight_upeer_bound),2)
            each_wk_k_weight[wk_idx,organization] = org_weight
            try:
                sum_of_weights[wk_idx]+=org_weight
            except:
                sum_of_weights[wk_idx]=org_weight
    for wk_idx in range(number_of_work_loads):
        print("for wkidx %s "%(wk_idx))
        list_of_number_of_flows.sort(reverse=True)
        
        for organization in range(organization_num):
            print("for organization %s "%(organization))
            each_flow_size_flows = {}
            for number_of_flows in list_of_number_of_flows:
                print("for # of flow set  %s "%(number_of_flows))
                selected_candidate_flows = []
                if number_of_flows == max(list_of_number_of_flows):
#                     print("flow set ",number_of_flows)
                    g = nx.Graph()
#                     print('[*] Loading topology...', topology_file)
                    try:
                        f = open(topology_file+".txt", 'r')
                    except:
                        f = open(topology_file, 'r')
                    header = f.readline()
                    nodes = []
                    flows = []
                    for line in f:
                        line = line.strip()
                        link = line.split('\t')
                        #print(line,link)
                        real_virtual = "real"
                        try:
                            i, s, d,  c,l,real_virtual = link
                        except:
                            i, s, d,  c,l = link  
                        g.add_edge(int(s),int(d),weight=1)
                        g.add_edge(int(d),int(s),weight=1)
#                     print("computing each flow shortest path length")
                    selected_real_nodes.sort()
                    selected_candidate_flows = []
                    for i in selected_real_nodes:
                        for j in selected_real_nodes:
                            if i!=j and (i,j) not in selected_candidate_flows and distance_between_two_nodes(g,i,j,virtual_links)<=hops_between_end_nodes:# and distance_between_two_nodes(g,i,j,virtual_links)<=7
                                selected_candidate_flows.append((i,j))
#                     print("all flows extracted and their path length")
                    each_flow_size_flows[number_of_flows] = selected_candidate_flows
                else:
#                     print("each_flow_size_upper_flow_size ",each_flow_size_upper_flow_size)
#                     print("each_flow_size_upper_flow_size[number_of_flows] ",number_of_flows,each_flow_size_upper_flow_size[number_of_flows])
                    selected_candidate_flows = each_flow_size_flows[each_flow_size_upper_flow_size[number_of_flows]]
                selected_flows = []
                sum_weights = 0
                each_flow_weight = {}
                each_flow_F_threshold = {}
                each_flow_max_rate_constraint ={}
                while(len(selected_flows)<(number_of_flows)):
                    random_flow = selected_candidate_flows[random.randint(0,len(selected_candidate_flows)-1)]
                    if random_flow not in selected_flows:
                        selected_flows.append(random_flow)
                        flow_weight = round(random.uniform(flow_weight_lower_boud,flow_weight_upeer_bound),2)
                        flow_max_rate_constraint = random.randint(10,1000)
                    
                        each_flow_max_rate_constraint[random_flow] = flow_max_rate_constraint
                        flow_F = round(random.uniform(lower_bound_Fth,upper_bound_Fth),2)
                        each_flow_weight[random_flow] = flow_weight
                        sum_weights = sum_weights+flow_weight
                        each_flow_F_threshold[random_flow] = flow_F
                        
                


                each_flow_size_flows[number_of_flows] = selected_flows
                if wk_idx==0  and organization ==0 and number_of_flows == max(list_of_number_of_flows):
                    print("we added the title")
                    work_load_file_lines = ["wk_indx,organization,weight,organization_F,user_pair,weight,flow_F,number_of_flows,flow_max_rate_constraint"+"\n"]
#                 else:
#                     print("we did not added title")
                #org_weight = round(random.uniform(0.1,1),2)
                org_weight = each_wk_k_weight[wk_idx,organization]/sum_of_weights[wk_idx]
                for flow in selected_flows:
#                     flow_weight = round(each_flow_weight[flow]/sum_weights,2)
                    flow_weight = round(each_flow_weight[flow],2)
                    print("adding line ",str(wk_idx)+","+str(organization)+","+str(org_weight))
                    flow_max_rate_constraint = each_flow_max_rate_constraint[flow]
                    work_load_file_lines.append(str(wk_idx)+","+str(organization)+","+str(org_weight)+","+str(1.0)+","+str(flow[0])+":"+str(flow[1])+","+str(flow_weight)+","+str(each_flow_F_threshold[flow])+","+str(number_of_flows)+","+str(flow_max_rate_constraint)+"\n")
    
    file_name = topology_file.split("data/")[1]
    file_name = topology_file.split(".txt")[0]
    with open(str(file_name)+"WK", "a+") as file_object:
        for line in work_load_file_lines:
            
            file_object.write(line)
#     with open(str(file_name)+"WK2", "a+") as file_object:
#         for line in WK2lines:
#             #print("line is ",line)
#             file_object.write(line)
    

In [4]:
distance_between_end_nodes = 12
each_flow_size_upper_flow_size={10:30,30:50,50:100}
set_of_flows = [50,30,10]
number_of_organziation = 3
lower_bound_Fth=0.94
upper_bound_Fth = 0.75
flow_weight_lower_boud= 0.3
flow_weight_upeer_bound = 0.7
# work_load_generator("Surfnet","data/ModifiedSurfnet_Eng_7hops_flowsv2.txt",30,number_of_organziation,set_of_flows,distance_between_end_nodes)
work_load_generator("Surfnet","data/ModifiedSurfnet_Eng_7hops_flowsv2_wider_flowsFths.txt",30,number_of_organziation,set_of_flows,distance_between_end_nodes)

# work_load_generator("data/2D.txt",100,3,[80])
# work_load_generator("data/ModifiedKdl.txt",10,3,[40,80])


[*] Loading topology... data/ModifiedSurfnet_Eng_7hops_flowsv2_wider_flowsFths.txt
for wkidx 0 
for organization 0 
for # of flow set  50 
we added the title
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.41216216216216217
adding line  0,0,0.412162162

adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.282442748091603
adding line  1,0,0.2

adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
adding line  2,0,0.38983050847457623
a

adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.3900709219858156
adding line  3,0,0.390070921

adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
adding line  4,0,0.24444444444444444
a

...

adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line  25,0,0.23404255319148934
adding line 

adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line  26,0,0.31496062992125984
adding line 

adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line  27,0,0.31578947368421056
adding line 

adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
adding line  28,0,0.3801169590643275
a

adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
adding line  29,0,0.3008130081300813
a

In [7]:
f = open("data/ModifiedSurfnetWK", 'r')
work_load_counter = 0
header = f.readline()
each_wk_k_flows = {}
work_loads = set([])
each_wk_k_flow_set_flow_weight = {}
for line in f:
    values = line.strip().split(',')#wk_indx,organization,weight,user_pair,weight
    wk_idx = int(values[0])
    k = int(values[1])
    org_weight = float(values[2])
    organization_F = float(values[3])
    i = int(values[4].split(":")[0])
    j = int(values[4].split(":")[1])
    flow_weight = float(values[5])
    flow_fidelity_threshold = float(values[6])
    user_pair = (i,j)
    flow_number_set = int(values[7])
    work_loads.add(wk_idx)
    
    each_wk_k_flow_set_flow_weight[wk_idx,k,flow_number_set,user_pair]=flow_weight
    try:
        each_wk_k_flows[wk_idx,k,flow_number_set].append(user_pair)
    except:
        each_wk_k_flows[wk_idx,k,flow_number_set]= [user_pair]
    
work_loads = list(work_loads)
print("having ",len(work_loads))
for wk_idx in work_loads:
    for k in [0,1,2]:
        flows_in_10 = each_wk_k_flows[wk_idx,k,10]
        flows_in_30 = each_wk_k_flows[wk_idx,k,30]
        flows_in_50 = each_wk_k_flows[wk_idx,k,50]
        print("# in 10 %s in 30 %s in 50 %s "%(len(flows_in_10),len(flows_in_30),len(flows_in_50)))
        for flow in flows_in_10: 
            in_10 = each_wk_k_flow_set_flow_weight[wk_idx,k,10,flow]
            in_30 = each_wk_k_flow_set_flow_weight[wk_idx,k,30,flow]
            in_50 = each_wk_k_flow_set_flow_weight[wk_idx,k,50,flow]
            print("in 10 is has weight %s in 30 %s 50 %s "%(in_10,in_30,in_50))
            if flow not in flows_in_30:
                print(" this is an Error since this flow should be in 30")
            if flow not in flows_in_50:
                print(" this is an Error since this flow should be in 50")
        
print("done")

having  30
# in 10 10 in 30 30 in 50 50 
in 10 is has weight 0.09 in 30 0.06 50 0.01 
in 10 is has weight 0.05 in 30 0.05 50 0.01 
in 10 is has weight 0.05 in 30 0.01 50 0.03 
in 10 is has weight 0.17 in 30 0.01 50 0.04 
in 10 is has weight 0.16 in 30 0.03 50 0.02 
in 10 is has weight 0.15 in 30 0.05 50 0.03 
in 10 is has weight 0.14 in 30 0.01 50 0.02 
in 10 is has weight 0.08 in 30 0.06 50 0.01 
in 10 is has weight 0.02 in 30 0.02 50 0.0 
in 10 is has weight 0.07 in 30 0.01 50 0.02 
# in 10 10 in 30 30 in 50 50 
in 10 is has weight 0.09 in 30 0.02 50 0.01 
in 10 is has weight 0.13 in 30 0.02 50 0.02 
in 10 is has weight 0.11 in 30 0.03 50 0.02 
in 10 is has weight 0.06 in 30 0.05 50 0.03 
in 10 is has weight 0.13 in 30 0.03 50 0.01 
in 10 is has weight 0.1 in 30 0.04 50 0.03 
in 10 is has weight 0.14 in 30 0.04 50 0.01 
in 10 is has weight 0.05 in 30 0.04 50 0.01 
in 10 is has weight 0.08 in 30 0.05 50 0.03 
in 10 is has weight 0.12 in 30 0.05 50 0.0 
# in 10 10 in 30 30 in 50 50 
in

In [ ]:
distance = 25
target_link_lenght = 15
edge = [0,1]
all_distances = []
real_links_rates = []
new_repeaters_id = 2
each_real_link_rate={}
each_real_link_lenght={}
edge_indx = 0
lines = []
real_edges = []
if int(distance)<=target_link_lenght:
    
    
    
    new_distance=distance
    c = 1
    etha = 10**(-0.1*0.2*new_distance)
    T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
    if T==0:
        T=1 * 10**(-6)
    alpha = 1
    all_distances.append(new_distance)
    edge_rate = round((2*c*etha*alpha)/T,3)
    real_links_rates.append(edge_rate)
    each_real_link_rate[(int(edge[0]),int(edge[1]))] = edge_rate
    each_real_link_rate[(int(edge[1]),int(edge[0]))] = edge_rate
    each_real_link_lenght[(int(edge[0]),int(edge[1]))] = new_distance
    each_real_link_lenght[(int(edge[1]),int(edge[0]))] = new_distance
    real_edges.append((int(edge[0]),int(edge[1])))

    print("unchanged link %s --> %s with length %s "%(edge[0],edge[1],new_distance))
    line = str(edge_indx)+"\t"+str(edge[0])+"\t"+str(edge[1])+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
    edge_indx+=1
    lines.append(line)

elif int(distance)<=2*target_link_lenght:
    print("we place only one repeater in the middle ")
    
else:
    print("%s ------------%s----------->%s "%(edge[0],distance,edge[1]))
    flag = False
    source = int(edge[0])
    end = new_repeaters_id
    for i in range(math.floor(int(distance)/dividing_link_metric)-1):
        flag = True
        new_distance = dividing_link_metric
        c = 1
        etha = 10**(-0.1*0.2*new_distance)
        T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
        if T==0:
            T=1 * 10**(-6)
        alpha = 1
        #print("T is %s for length %s"%(T,distance))


        all_distances.append(new_distance)
        edge_rate = round((2*c*etha*alpha)/T,3)
        real_links_rates.append(edge_rate)
        each_real_link_rate[(source,end)] = edge_rate
        each_real_link_rate[(end,source)] = edge_rate
        each_real_link_lenght[(source,end)] = new_distance
        each_real_link_lenght[(end,source)] = new_distance
        real_edges.append((source,end))

        print("1. %s --> %s with length %s "%(source,end,new_distance))
        line = str(edge_indx)+"\t"+str(source)+"\t"+str(end)+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
        edge_indx+=1
        lines.append(line)
        source = end
        new_repeaters_id+=1
        end = new_repeaters_id


    if int(distance)%dividing_link_metric!=0:


        if not flag:
            source = int(edge[0])
            end = new_repeaters_id
            new_distance = dividing_link_metric
            c = 1
            etha = 10**(-0.1*0.2*new_distance)
            T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
            if T==0:
                T=1 * 10**(-6)
            alpha = 1
            #print("T is %s for length %s"%(T,distance))


            all_distances.append(new_distance)
            edge_rate = round((2*c*etha*alpha)/T,3)
            real_links_rates.append(edge_rate)
            each_real_link_rate[(source,end)] = edge_rate
            each_real_link_rate[(end,source)] = edge_rate
            each_real_link_lenght[(source,end)] = new_distance
            each_real_link_lenght[(end,source)] = new_distance
            real_edges.append((source,end))
            print("1. %s --> %s with length %s "%(source,end,new_distance))
            line = str(edge_indx)+"\t"+str(source)+"\t"+str(end)+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
            edge_indx+=1
            lines.append(line)

            source = end
            end = new_repeaters_id

            source = end

            end = int(edge[1])
            new_distance = distance%dividing_link_metric
            c = 1
            etha = 10**(-0.1*0.2*new_distance)
            T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
            if T==0:
                T=1 * 10**(-6)
            alpha = 1
            #print("T is %s for length %s"%(T,distance))


            all_distances.append(new_distance)
            edge_rate = round((2*c*etha*alpha)/T,3)
            real_links_rates.append(edge_rate)
            each_real_link_rate[(source,int(edge[1]))] = edge_rate
            each_real_link_rate[(int(edge[1]),source)] = edge_rate
            each_real_link_lenght[(source,int(edge[1]))] = new_distance
            each_real_link_lenght[(int(edge[1]),source)] = new_distance
            real_edges.append((source,int(edge[1])))
            print("22 second extra %s --> %s with length %s "%(source,edge[1],new_distance))
            line = str(edge_indx)+"\t"+str(source)+"\t"+str(edge[1])+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
            edge_indx+=1
            lines.append(line)

            new_repeaters_id = new_repeaters_id+1

        else:


            source = new_repeaters_id-1
            end = new_repeaters_id
            new_distance = dividing_link_metric
            c = 1
            etha = 10**(-0.1*0.2*new_distance)
            T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
            if T==0:
                T=1 * 10**(-6)
            alpha = 1
            #print("T is %s for length %s"%(T,distance))


            all_distances.append(new_distance)
            edge_rate = round((2*c*etha*alpha)/T,3)
            real_links_rates.append(edge_rate)
            each_real_link_rate[(source,end)] = edge_rate
            each_real_link_rate[(end,source)] = edge_rate
            each_real_link_lenght[(source,end)] = new_distance
            each_real_link_lenght[(end,source)] = new_distance
            real_edges.append((source,end))
            print("first extra %s --> %s with length %s "%(source,end,new_distance))
            line = str(edge_indx)+"\t"+str(source)+"\t"+str(end)+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
            edge_indx+=1
            lines.append(line)

            source = end

            end = int(edge[1])
            new_distance = distance%dividing_link_metric
            c = 1
            etha = 10**(-0.1*0.2*new_distance)
            T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
            if T==0:
                T=1 * 10**(-6)
            alpha = 1
            #print("T is %s for length %s"%(T,distance))


            all_distances.append(new_distance)
            edge_rate = round((2*c*etha*alpha)/T,3)
            real_links_rates.append(edge_rate)
            each_real_link_rate[(source,int(edge[1]))] = edge_rate
            each_real_link_rate[(int(edge[1]),source)] = edge_rate
            each_real_link_lenght[(source,int(edge[1]))] = new_distance
            each_real_link_lenght[(int(edge[1]),source)] = new_distance
            real_edges.append((source,int(edge[1])))
            print("second extra %s --> %s with length %s "%(source,edge[1],new_distance))
            line = str(edge_indx)+"\t"+str(source)+"\t"+str(edge[1])+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
            edge_indx+=1
            lines.append(line)


            new_repeaters_id = new_repeaters_id+1
    else:
        new_distance = dividing_link_metric
        c = 1
        etha = 10**(-0.1*0.2*new_distance)
        T = (new_distance*10**(-4))/25# based on simulation setup of data link layer paper
        if T==0:
            T=1 * 10**(-6)
        alpha = 1
        #print("T is %s for length %s"%(T,distance))

        end = new_repeaters_id-1
        all_distances.append(new_distance)
        edge_rate = round((2*c*etha*alpha)/T,3)
        real_links_rates.append(edge_rate)
        each_real_link_rate[(end,int(edge[1]))] = edge_rate
        each_real_link_rate[(int(edge[1]),end)] = edge_rate
        each_real_link_lenght[(end,int(edge[1]))] = new_distance
        each_real_link_lenght[(int(edge[1]),end)] = new_distance
        real_edges.append((end,int(edge[1])))
        print("third %s -> %s with length %s"%(end,int(edge[1]),new_distance))
        line = str(edge_indx)+"\t"+str(end)+"\t"+str(edge[1])+"\t"+str(edge_rate)+"\t"+str(new_distance)+"\t"+"real"+"\n"
        edge_indx+=1
        lines.append(line)
    
        
print("new_repeaters_id ",new_repeaters_id)

In [ ]:
print(lines)

In [ ]:
distance = 56
print(distance%30)
if distance%30!=0:
    print("add two")
for i in range(math.floor(distance/30)-1):
    print(i)

In [ ]:
def split(x, n):
    points = []
    # If we cannot split the
    # number into exactly 'N' parts
    if(x < n):
        print(-1)
 
    # If x % n == 0 then the minimum
    # difference is 0 and all
    # numbers are x / n
    elif (x % n == 0):
        for i in range(n):
            print(x//n, end =" ")
            points.append(x//n)
    else:
        # upto n-(x % n) the values
        # will be x / n
        # after that the values
        # will be x / n + 1
        zp = n - (x % n)
        pp = x//n
        for i in range(n):
            if(i>= zp):
                print(pp + 1, end =" ")
                points.append(pp + 1)
            else:
                print(pp, end =" ")
                points.append(pp)
    return points
       
# Driver code         
x = 30
each_repeater_placement = 10
n = int(math.ceil(x/each_repeater_placement))
list_points = split(x, n)
print(list_points)
next_point = 10
flag = True
edge = (0,1)
for indx,point in enumerate(list_points):
#     print(indx,point)
    if flag:
        flag = False
        source = edge[0]
        end = next_point
        print("connecting %s --> %s length %s"%(source,end,point))
        next_point+=1
    else:
        if indx== len(list_points)-1:
            source = end
            end = edge[1]
            print("connecting %s --> %s length %s"%(source,end,point))
            next_point+=1
        else:
            source = end
            end = next_point
            print("connecting %s --> %s length %s"%(source,end,point))
            next_point+=1
